# Week 2 Exercises

In this weeks exercises you will use Numpy/Scipy to impliment some numerical algorithms and then you will use Pandas to perform a rudamentary data analysis using the KDD 98 dataset.  Along the way you will use unix/basic python from the first week as well as git to save your work.

As a first step we import the libraries we'll use later on.  This allows us to use numpy library calls by prefixing the call with np.

In [1]:
#Import the libraries 
import numpy as np
import scipy as sp
import pandas as pd

## Matrix Manipulations
Lets first create a matrix and perform some manipulations of it.

Using numpy's matrix data structure, define the following matricies:

$$A=\left[ \begin{array}{ccc} 3 & 5 & 9 \\ 3 & 3 & 4 \\ 5 & 9 & 17 \end{array} \right]$$

$$B=\left[ \begin{array}{c} 2 \\ 1 \\ 4 \end{array} \right]$$

After this solve the matrix equation:
$$Ax = B$$

Now write three functions for matrix multiply $C=AB$ in each of the following styles:

1. By using nested for loops to impliment the naive algorithm ($C_{ij}=\sum_{k=0}^{m-1}A_{ik}B_{kj}$)
2. Using numpy's built in martrix multiplication  
3. Using Cython

The three methods should have the same answer

In [2]:
A = [[3,5,9],[3,3,4],[5,9,17]]
B = [[2],[1],[4]]

In [3]:
def python_multiply(a,b):
    row_a = len(a)
    col_a = len(a[0])
    row_b = len(b)
    col_b = len(b[0])
    
    if col_a != row_b:
        print "Incorrect dimensions considered. Check the dimensions."
        return
    
    result = [[0 for row in range(col_b)] for col in range(row_a)]
    
    for i in range(row_a):
        for j in range(col_b):
            for k in range(col_a):
                result[i][j] += a[i][k]*b[k][j]
    print 'The resulting matrix is \n', np.array(result)
    
    
python_multiply(A,B)

The resulting matrix is 
[[47]
 [25]
 [87]]


In [4]:
def numpy_multiply(a,b):
    a = np.array(a)
    b = np.array(b)
    print 'The resulting matrix is \n', np.mat(a) * np.mat(b)

numpy_multiply(A,B)

The resulting matrix is 
[[47]
 [25]
 [87]]


In [5]:
%load_ext Cython

In [6]:
%%cython
cimport numpy as np
import numpy as np

def cython_multiply(np.ndarray[np.float_t, ndim=2] a, np.ndarray[np.float64_t, ndim=2] b):
    cdef np.ndarray[np.float64_t, ndim=2] result
    cdef long i, j, k
        if a.shape[1] != b.shape[0]:
            print 'Incorrect dimensions considered. Check the dimensions.'
            return None
    
    result = np.zeros((a.shape[0], b.shape[1]))
    for i in range(a.shape[0]):
        for j in range(b.shape[1]):
            for k in range(a.shape[1]):
                result[i, j] += a[i, k]*b[k, j]

    print 'The resulting matrix is \n', np.array(result)

cython_multiply(A,B)    


Error compiling Cython file:
------------------------------------------------------------
...
import numpy as np

def cython_multiply(np.ndarray[np.float_t, ndim=2] a, np.ndarray[np.float64_t, ndim=2] b):
    cdef np.ndarray[np.float64_t, ndim=2] result
    cdef long i, j, k
        if a.shape[1] != b.shape[0]:
^
------------------------------------------------------------

C:\Users\Orysya\.ipython\cython\_cython_magic_14f51fcd743a13dff27aad9da2f409b7.pyx:7:0: Possible inconsistent indentation

Error compiling Cython file:
------------------------------------------------------------
...
import numpy as np

def cython_multiply(np.ndarray[np.float_t, ndim=2] a, np.ndarray[np.float64_t, ndim=2] b):
    cdef np.ndarray[np.float64_t, ndim=2] result
    cdef long i, j, k
        if a.shape[1] != b.shape[0]:
^
------------------------------------------------------------

C:\Users\Orysya\.ipython\cython\_cython_magic_14f51fcd743a13dff27aad9da2f409b7.pyx:7:0: Expected an identifier or literal


Now we wish to evaluate the performance of these three methods.  Write a method that given three dmiensions (a,b,c) makes a random a x b and b x c matrix and computes the product using your three functions and reports the speed of each method.

After this measure performance of each method for all $a,b,c \in \{10,100,1000,10000\}$ and plot the results.  Is one method always the fastest?  Discuss why this is or is not the case.

In [8]:
def performance(a,b,c):
    mat_A = np.random.choice([x for x in xrange(0,10)], a*b)
    mat_A.resize(a, b)
    
    mat_B = np.random.choice([x for x in xrange(0,10)], b*c)
    mat_B.resize(b,c)
    
    %time python_multiply(mat_A, mat_B)
    %time numpy_multiply(mat_A, mat_B)
    #%time cython_multiply(mat_A, mat_B)

In [9]:
from itertools import permutations

def measure(dimension_list, size):
    combos = []
    for i in permutations(dimension_list, size):
    #How do I generate permutations of length LEN given a list of N Items?
        combos.append(i)

    for i in combos:
        print performance(i[0],i[1],i[2])
        

dims = [10,100,1000,10000]

measure(dims, 3)

The resulting matrix is 
[[2159 2257 2048 ..., 2470 2162 2199]
 [1877 2086 1764 ..., 2006 2051 1950]
 [1811 2107 1764 ..., 2056 1910 1892]
 ..., 
 [1564 1993 1753 ..., 2066 1884 1872]
 [1797 2089 1718 ..., 2116 1977 1794]
 [1766 1989 1623 ..., 2068 1821 1750]]
Wall time: 1.62 s
The resulting matrix is 
[[2159 2257 2048 ..., 2470 2162 2199]
 [1877 2086 1764 ..., 2006 2051 1950]
 [1811 2107 1764 ..., 2056 1910 1892]
 ..., 
 [1564 1993 1753 ..., 2066 1884 1872]
 [1797 2089 1718 ..., 2116 1977 1794]
 [1766 1989 1623 ..., 2068 1821 1750]]
Wall time: 3 ms
None
The resulting matrix is 
[[1996 1913 2000 ..., 1727 2189 1830]
 [2071 1897 2053 ..., 1937 2351 1783]
 [2055 1989 2089 ..., 1908 2299 1837]
 ..., 
 [2147 2173 2213 ..., 2229 2482 2029]
 [2223 2040 2352 ..., 2039 2443 2008]
 [2036 1972 2302 ..., 2016 2352 1858]]
Wall time: 20.1 s
The resulting matrix is 
[[1996 1913 2000 ..., 1727 2189 1830]
 [2071 1897 2053 ..., 1937 2351 1783]
 [2055 1989 2089 ..., 1908 2299 1837]
 ..., 
 [2147 2173 22

In [ ]:
## need to complete the Cython section + plotting

**BONUS** Now repeat the past two problems but instead of computing the matrix product, compute a matrix's [determinant](http://en.wikipedia.org/wiki/Determinant).  Measure performance for matricies of various sizes and discuss the results.  Determinant may get impractical to calculate for not too huge of matricies, so no need to goto 1000x1000 matricies.

### IO Exercises

Below is a map of various datatypes in python that you have come across and their corresponding JSON equivalents.

$$Datatypes=\left[ \begin{array}{cc} JSON & Python3 \\ object & dictionary \\ array & list \\ string & string \\ integer	& integer \\ real number & float \\ true & True \\ false & False \\ null & None  \end{array} \right]$$


There are atleast two very important python datatypes missing in the above list. 
Can you find the same?  [list the two mising python datatypes in this markdown cell below]

1. ###set
2. ###tuple

Now We can save the above map as a dictionary with Key-value pairs 
1. create a python dictionary named dataypes, having the above map as the Key-value pairs with Python datatypes as values and JSON equivalents as keys.
2. Save it as a pickle called datatypes and gzip the same.
3. Reload this pickle, and read the file contents and output the data in the following formatted way as given in this example - "The JSON equivalent for the Python datatype Dictionary is Object". Output similarly for the rest of the key-value pairs.
4. Save this data as a JSON but using Python datatypes as keys and JSON equivalent as values this time. 

In [11]:
import pickle
import json
import gzip

datatypes = {'object': 'dictionary', 'array':'list', 'string':'string', 'integer': 'integer', 'realnumber':'float', 'true':'True', 'false':'False', 'null':'None'}

pickle.dump(datatypes, gzip.open('datatypeszip.pkl', 'wb'))
data = pickle.load(gzip.open('datatypeszip.pkl', 'rb'))
datajson = {}

for key, value in data.items():
    print 'The JSON equivalent for the Python datatype %s is %s' % (value, key)
    datajson[value] = key

datajson
json.dump(datajson, gzip.open('datatypeszip.jsn', 'wb'))

The JSON equivalent for the Python datatype False is false
The JSON equivalent for the Python datatype string is string
The JSON equivalent for the Python datatype dictionary is object
The JSON equivalent for the Python datatype integer is integer
The JSON equivalent for the Python datatype list is array
The JSON equivalent for the Python datatype None is null
The JSON equivalent for the Python datatype True is true
The JSON equivalent for the Python datatype float is realnumber


## Pandas Data Analysis
Pandas gives us a nice set of tools to work with columnar data (similar to R's dataframe). 
To learn how to use this it makes the most sense to use a real data set.
For this assignment we'll use the KDD Cup 1998 dataset, which can be sourced from http://kdd.ics.uci.edu/databases/kddcup98/kddcup98.html .


### Acquiring Data
First we pull the README file from the dataset into this notebook via the unix "curl" command.  Remember you can hide/minimize output cells via the button on the left of the output.

In [2]:
!curl http://kdd.ics.uci.edu/databases/kddcup98/epsilon_mirror/readme

+--------------------------------------------------------------------+
| NOTE TO ALL DOWN-LOADERS                                           |
+--------------------------------------------------------------------+

The KDD-CUP-98 data set and the accompanying documentation are now 
available for general use with the following restrictions: 

  (1) The users of the data must notify 

	Ismail Parsa	(iparsa@epsilon.com) and
	Ken Howes	(khowes@epsilon.com) 

  in the event they produce results, visuals or tables, etc. from the 
  data and send a note that includes a summary of the final result. 

  (2) The authors of published and/or unpublished articles that use 
  the KDD-Cup-98 data set must also notify the individuals listed 
  above and send a copy of their published and/or unpublished work. 

  (3) If you intend to use this data set for training or educational
  purposes, you must not reveal the name of the sponsor PVA 
  (Paralyzed Veterans of America) to the trainees or students. Yo

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  4134  100  4134    0     0  43978      0 --:--:-- --:--:-- --:--:-- 43978


In [ ]:
#git status to make sure that untracked file

As you can see this README describes several files which may be of use.  In particular there are two more documentation files (DOC and DIC) we should read to get an idea of the data format.  Bring these files into the notebook.

In [3]:
!curl http://kdd.ics.uci.edu/databases/kddcup98/epsilon_mirror/cup98doc.txt

EPSILON CONFIDENTIAL      EPSILON CONFIDENTIAL    EPSILON CONFIDENTIAL

    INFORMATION LISTED BELOW IS AVAILABLE UNDER THE TERMS OF THE  
                      CONFIDENTIALITY AGREEMENT                

EPSILON CONFIDENTIAL      EPSILON CONFIDENTIAL    EPSILON CONFIDENTIAL

+--------------------------------------------------------------------+
|                   DOCUMENTATION TO ACCOMPANY                       |
|                                                                    |
|                          KDD-CUP-98                                |
|                                                                    |
|          The Second International Knowledge Discovery and          |
|                 Data Mining Tools Competition                      |
|                                                                    |
|                Held in Conjunction with KDD-98                     |
|                                                                    |
|          The

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 62956  100 62956    0     0   391k      0 --:--:-- --:--:-- --:--:--  391k


In [4]:
!curl http://kdd.ics.uci.edu/databases/kddcup98/epsilon_mirror/cup98dic.txt



EPSILON CONFIDENTIAL      EPSILON CONFIDENTIAL    EPSILON CONFIDENTIAL

    INFORMATION LISTED BELOW IS AVAILABLE UNDER THE TERMS OF THE  
                      CONFIDENTIALITY AGREEMENT                

EPSILON CONFIDENTIAL      EPSILON CONFIDENTIAL    EPSILON CONFIDENTIAL

+--------------------------------------------------------------------+
|                PARALYZED VETERANS OF AMERICA (PVA)                 |
|                  DATA DICTIONARY TO ACCOMPANY                      |
|                                                                    |
|                          KDD-CUP-98                                |
|                                                                    |
|          The Second International Knowledge Discovery and          |
|                 Data Mining Tools Competition                      |
|                                                                    |
|                Held in Conjunction with KDD-98                     |
|           

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 56987  100 56987    0     0   323k      0 --:--:-- --:--:-- --:--:--  323k


Now we wish to download the cup98lrn.zip file and unzip it into a new subdirectory called "data".  
However, since this file is pretty big we don't want to store it on github.  
Luckily git provides the [.gitignore](http://git-scm.com/docs/gitignore) file which allows us to specify files we don't want to put into our git repository.

Please do the following steps:

1. Add the directory "data" to the .gitignore file
2. Commit the new .gitignore file
3. Create a new directory "data"
4. Download http://kdd.ics.uci.edu/databases/kddcup98/epsilon_mirror/cup98lrn.zip into the data directory
5. Unzip the cup98lrn.zip (we will only be using the unzipped version, so feel free to remove the zip file)
6. Run "git status" to show that the data directory is not an untracked file (this indicates it is ignored)

**NOTE:** These steps only need to be run once, it is advised you comment all the lines out by putting a # at the start of each line after they have run.  This will save you time in the future when you have to rerun all cells/don't want to spend a few minutes downloading the data file.

Now perform some basic sanity checks on the data.  Using a combination of unix/basic python answer the following questions:

1. How many lines are there?  
2. Is the file character seperated or fixed width format?
3. Is there a header?  If so how many fields are in it?
4. Do all rows have the same number of fields as the header?
5. Does anyhting in 1-4 disagree with the readme file or indicate erroneous data?

Give answers to questions 1-4 in this markdown cell:

1. 
2. 
3. 
4. 

Now load the data file into a pandas data frame called "learn".  To save some time, we've loaded the data dictionary into col_types.  

Finally split learn into two data frames, learn_y: the targets (two columns described in the documentation) and learn_x: the predictors (everything but the targets)

In [ ]:
dict_file = open("dict.dat")
col_types = [ (x.split("\t")[0], x.strip().split("\t")[1]) for x in dict_file.readlines() ]

### Summarizing Data
Now that we have loaded data into the learn table, we wish to to summarize the data.  
Write a function called summary which takes a pandas data frame and prints a summary of each column containing the following:

If the column is numeric:

1. Mean
2. Standard Deviation
3. Min/Max
4. Number of missing values (NaN, Inf, NA)

If the column is non numeric:

1. Number of distinct values
2. Number of missing values (NaN, INF, NA, blank/all spaces)
3. The frequency of the 3 most common values and 3 least common values

Format the output to be human readable.

For example:
> Field_1  
> mean: 50  
> std_dev: 25  
> min: 0  
> max: 100  
> missing: 5
>  
> Field_2  
> distinct_values: 100  
> missing: 10  
>  
> 3 most common:  
>   the: 1000  
>   cat: 950  
>   meows: 900  
>  
> 3 least common:  
>   dogs: 5  
>   lizards: 4  
>   eggs: 1  

 ### Pandas analysis on Calit2 data 

Import data from http://archive.ics.uci.edu/ml/machine-learning-databases/event-detection/CalIt2.data using curl

This data comes from the main door of the CalIt2 building at UCI. Observations come from 2 data streams (people flow in and out of the building), over 15 weeks, 48 time slices per day (half hour count aggregates).

Attribute Information:
1. Flow ID: 7 is out flow, 9 is in flow
2. Date: MM/DD/YY
3. Time: HH:MM:SS
4. Count: Number of counts reported for the previous half hour


In [5]:
!curl http://archive.ics.uci.edu/ml/machine-learning-databases/event-detection/CalIt2.data > Calit2.txt
!cat Calit2.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  218k  100  218k    0     0   872k      0 --:--:-- --:--:-- --:--:--  872k


7,07/24/05,00:00:00,0
9,07/24/05,00:00:00,0
7,07/24/05,00:30:00,1
9,07/24/05,00:30:00,0
7,07/24/05,01:00:00,0
9,07/24/05,01:00:00,0
7,07/24/05,01:30:00,0
9,07/24/05,01:30:00,0
7,07/24/05,02:00:00,0
9,07/24/05,02:00:00,0
7,07/24/05,02:30:00,2
9,07/24/05,02:30:00,0
7,07/24/05,03:00:00,0
9,07/24/05,03:00:00,0
7,07/24/05,03:30:00,0
9,07/24/05,03:30:00,0
7,07/24/05,04:00:00,0
9,07/24/05,04:00:00,0
7,07/24/05,04:30:00,0
9,07/24/05,04:30:00,0
7,07/24/05,05:00:00,0
9,07/24/05,05:00:00,0
7,07/24/05,05:30:00,0
9,07/24/05,05:30:00,0
7,07/24/05,06:00:00,0
9,07/24/05,06:00:00,0
7,07/24/05,06:30:00,0
9,07/24/05,06:30:00,0
7,07/24/05,07:00:00,0
9,07/24/05,07:00:00,0
7,07/24/05,07:30:00,2
9,07/24/05,07:30:00,3
7,07/24/05,08:00:00,0
9,07/24/05,08:00:00,0
7,07/24/05,08:30:00,0
9,07/24/05,08:30:00,0
7,07/24/05,09:00:00,0
9,07/24/05,09:00:00,1
7,07/24/05,09:30:00,1
9,07/24/05,09:30:00,1
7,07/24/05,10:00:00,0
9,07/24/05,10:00:00,0
7,07/24/05,10:30:00,0
9,07/24/05,10:30:00,0
7,07/24/05,11:00:00,1
9,07/24/05

In [16]:
data = pd.read_csv('C:/Users/Orysya/Documents/DSE/DSE200-notebooks/day_2_github_io_numpy_pandas//Calit2.txt', header = None)
data.columns=['Flow', 'Date', 'Time', 'Count']
data.head()

,Flow,Date,Time,Count
0,7,07/24/05,00:00:00,0
1,9,07/24/05,00:00:00,0
2,7,07/24/05,00:30:00,1
3,9,07/24/05,00:30:00,0
4,7,07/24/05,01:00:00,0


#### Selecting Data ####
1. Select all data for the date July 24 2005 having flow id=7. Also output the row count of results 
2. Select all rows whose count is greater than 5. Sort the result on count in descending order and output the top 10 rows

In [21]:
#for 1:
date = data['Date'].isin(['07/24/05'])
flow_7 = data['Flow'] == 7
output = data[date & flow_7]
output

,Flow,Date,Time,Count
0,7,07/24/05,00:00:00,0
2,7,07/24/05,00:30:00,1
4,7,07/24/05,01:00:00,0
6,7,07/24/05,01:30:00,0
8,7,07/24/05,02:00:00,0
10,7,07/24/05,02:30:00,2
12,7,07/24/05,03:00:00,0
14,7,07/24/05,03:30:00,0
16,7,07/24/05,04:00:00,0
18,7,07/24/05,04:30:00,0


In [22]:
print len(output.index)

48


In [32]:
#for 2:
count_5 = data[data['Count'] > 5]
count_5 = count_5.sort(columns='Count', ascending=False)
top10 = count_5.head(10)
top10

,Flow,Date,Time,Count
6879,9,10/03/05,15:30:00,62
335,9,07/27/05,11:30:00,61
6881,9,10/03/05,16:00:00,55
9686,7,11/01/05,21:30:00,54
9682,7,11/01/05,20:30:00,54
9681,9,11/01/05,20:00:00,53
6878,7,10/03/05,15:30:00,52
9933,9,11/04/05,11:00:00,50
8913,9,10/24/05,20:00:00,50
3615,9,08/30/05,15:30:00,50


#### Apply function ####
1. For the 10 rows outputted above, use Pandas Apply function to subtract lowest value of the 10 from all of them and then output the average value of the resulting counts
2. On the entire data, use apply function to sum all counts with flow_id=9 and date is 07/24/05

In [45]:
#for 1:
print (top10['Count'].apply(lambda x: x-min(top10['Count']))).mean()
top10['Adjusted_average'] = (top10['Count'].apply(lambda x: x-min(top10['Count']))).mean()
top10

4.1


C:\Users\Orysya\Anaconda\lib\site-packages\IPython\kernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Flow,Date,Time,Count,Adjusted_average
6879,9,10/03/05,15:30:00,62,4.1
335,9,07/27/05,11:30:00,61,4.1
6881,9,10/03/05,16:00:00,55,4.1
9686,7,11/01/05,21:30:00,54,4.1
9682,7,11/01/05,20:30:00,54,4.1
9681,9,11/01/05,20:00:00,53,4.1
6878,7,10/03/05,15:30:00,52,4.1
9933,9,11/04/05,11:00:00,50,4.1
8913,9,10/24/05,20:00:00,50,4.1
3615,9,08/30/05,15:30:00,50,4.1


In [65]:
#for 2:
flow_9 = data['Flow'] == 9
date = data['Date'].isin(['07/24/05'])
new = data[flow_9 & date]
print new.apply(np.sum)['Count']

16


#### Indexing an Selecting ####
Explain the following

1. loc: returns the following indexes df.loc[inclusive:inclusive]; only works on index
2. iloc: returns the following indexes df.iloc[inclusive:exclusive] and should only be used to refer to indexes as positions; only works on position
3. ix: can retrieve data from a dataframe without it being in the index
4. at: can retrieve scalar values, it works as a fast loc
5. iat: can retrieve scalar values, it works as a fast iloc

Highlight the differences by providing usecases where one is more useful than the other


Write a function to take two dates as input and return all flow ids and counts in that date range having both the dates inclusive. You can use pandas to_datetime function to convert the date to pandas datetime format 

#### Grouping ####
1. Select data in the month of August 2005 having flow id=7
2. Group the data based on date and get the max count per date

In [118]:
data['Date'] = pd.to_datetime(data['Date'])
august = data[data['Date'].apply(lambda x: x.strftime('%B-%Y')) == 'August-2005']
group = august[august['Flow'] == 7]
group

,Flow,Date,Time,Count
768,7,2005-08-01,00:00:00,1
770,7,2005-08-01,00:30:00,0
772,7,2005-08-01,01:00:00,0
774,7,2005-08-01,01:30:00,0
776,7,2005-08-01,02:00:00,0
778,7,2005-08-01,02:30:00,0
780,7,2005-08-01,03:00:00,0
782,7,2005-08-01,03:30:00,0
784,7,2005-08-01,04:00:00,0
786,7,2005-08-01,04:30:00,0


In [123]:
A = group.groupby('Date')
A.apply(np.max)['Count']
#should I include 'Flow' and 'Time' ?????

Date
2005-08-01    18
2005-08-02    16
2005-08-03    26
2005-08-04    40
2005-08-05    17
2005-08-06     8
2005-08-07     3
2005-08-08    16
2005-08-09    19
2005-08-10    25
2005-08-11    16
2005-08-12    21
2005-08-13     5
2005-08-14     3
2005-08-15    17
2005-08-16    24
2005-08-17    22
2005-08-18    39
2005-08-19    23
2005-08-20     3
2005-08-21     4
2005-08-22    23
2005-08-23    25
2005-08-24    17
2005-08-25    24
2005-08-26    22
2005-08-27     3
2005-08-28     7
2005-08-29    15
2005-08-30    47
2005-08-31    19
Name: Count, dtype: int64

#### Stacking, Unstacking ####
1. Stack the data with count and flow_id as indexes
2. Use reset_index to reset the stacked hierarchy by 1 level. The index then will just be the counts
3. Unstack the data to get back original data

In [150]:
#for 1:
stack1 = data.set_index(['Count', 'Flow']).stack()
print stack1
#for 2:
stack2 = stack1.reset_index(level=1)
print stack2.head()
#for 3:




Count  Flow      
0      7     Date   2005-07-24 00:00:00
             Time   2016-10-22 00:00:00
       9     Date   2005-07-24 00:00:00
             Time   2016-10-22 00:00:00
1      7     Date   2005-07-24 00:00:00
             Time   2016-10-22 00:30:00
0      9     Date   2005-07-24 00:00:00
             Time   2016-10-22 00:30:00
       7     Date   2005-07-24 00:00:00
             Time   2016-10-22 01:00:00
       9     Date   2005-07-24 00:00:00
             Time   2016-10-22 01:00:00
       7     Date   2005-07-24 00:00:00
             Time   2016-10-22 01:30:00
       9     Date   2005-07-24 00:00:00
             Time   2016-10-22 01:30:00
       7     Date   2005-07-24 00:00:00
             Time   2016-10-22 02:00:00
       9     Date   2005-07-24 00:00:00
             Time   2016-10-22 02:00:00
2      7     Date   2005-07-24 00:00:00
             Time   2016-10-22 02:30:00
0      9     Date   2005-07-24 00:00:00
             Time   2016-10-22 02:30:00
       7     Date   20

#### Pandas and Matplotlib

Plot a histogram of date vs total counts for flow_id=7 and flow_id=9 for the month of July 2005

In [224]:
%matplotlib inline
data['Date'] = pd.to_datetime(data['Date'])
july = data[data['Date'].apply(lambda x: x.strftime('%B-%Y')) == 'July-2005']
group = july[july['Flow'] == 9]
B = group.groupby('Date').sum()

In [225]:
timeCount_df = B.['Date','Count']
timeCount_df['Time'] = timeCount_df['Time'].apply(lambda x: x[:5])
ax = timeCount_df.plot(x='Time', y='Count', kind='bar', legend=False, figsize=(20, 10), title='July Entries to CALIT2')
ax.yaxis.set_ticks(np.arange(0, 5, 1.0))
ax.set_ylabel("# of Visitors")

SyntaxError: invalid syntax (<ipython-input-225-e8e2522c5cb0>, line 1)